In [19]:
import io
import base64
import requests
import json 
from PIL import Image,ImageDraw
import hashlib
from dotenv import load_dotenv
import re

In [20]:
import os
# Cargar las variables de entorno desde el archivo .env
load_dotenv()
# Acceder a las variables de entorno
ENDPOINT_KEY = os.getenv('ENDPOINT_KEY')

In [33]:
def parse_coordinates(response_text):
    
    if response_text == "Bottle:[]":
        return {"Image": []}
    
    pattern = r"Bottle (\d+): upper=\[(\d+), (\d+)\], lower=\[(\d+), (\d+)\]"
    matches = re.findall(pattern, response_text)

    coordinates = {
        "Image": [
            {
                "bottle": int(match[0]),
                "coordinates": {
                    "upper": [int(match[1]), int(match[2])],
                    "lower": [int(match[3]), int(match[4])]
                }
            } for match in matches
        ]
    }
    return coordinates

In [22]:
def convert_image(image_path):
    with Image.open(image_path) as img:
        buffer = io.BytesIO()
        if img.format not in ['PNG', 'JPEG', 'GIF', 'WEBP']:
            img.save(buffer, format="PNG")
            extension = "png"
        else:
            img.save(buffer, format=img.format)
            extension = img.format.lower()
        
        processed_image_path = f"processed_image.{extension}"
            
            
        buffer.seek(0)  
        return buffer.getvalue(),processed_image_path, extension
    

def calculate_checksum(image_data):
    return hashlib.md5(image_data).hexdigest()

In [23]:
def draw_lines_on_image_base64(encoded_image, coordinates):
    image_data = base64.b64decode(encoded_image)
    img = Image.open(io.BytesIO(image_data))
    
    draw = ImageDraw.Draw(img)
    for bottle in coordinates['Image']:
        upper = tuple(bottle['coordinates']['upper'])
        lower = tuple(bottle['coordinates']['lower'])
        draw.line([upper, lower], fill='red', width=8)
    
    result_image_path = "result_image.png"
    img.save(result_image_path)
    return result_image_path

In [35]:
# Path to your image file
image_path = "img5.jpg"

# Convert the image
image_data, image_path, extension = convert_image(image_path)

checksum = calculate_checksum(image_data)

print("Checksum:", checksum)

encoded_image = base64.b64encode(image_data).decode('utf-8')
     
endpoint = f"https://plvf1e97ll.execute-api.us-east-1.amazonaws.com/production/deteccion" 
    
headers = {
        "Content-Type": "application/json",
        "x-api-key": ENDPOINT_KEY
    } 

inputRoot = {
    "Checksum": checksum,
    "Extension": extension,
    "Image": encoded_image 
} 

dumbs = json.dumps(inputRoot)

response = requests.post(endpoint, headers=headers, data=dumbs)
    

print("Solicitud recibida del servidor")
server_response = response.json()
print("Respuesta del servidor:", server_response)
          

Checksum: 73632338370f6a5a4ac5e3d7a7b78834
Solicitud recibida del servidor
Respuesta del servidor: {'statusCode': 200, 'body': '"Bottle 1: upper=[101, 90], lower=[140, 170]\\nBottle 2: upper=[141, 90], lower=[180, 170]\\nBottle 3: upper=[181, 90], lower=[220, 170]\\nBottle 4: upper=[221, 90], lower=[260, 170]\\nBottle 5: upper=[261, 90], lower=[300, 170]\\nBottle 6: upper=[301, 90], lower=[340, 170]\\nBottle 7: upper=[341, 90], lower=[380, 170]\\nBottle 8: upper=[381, 90], lower=[420, 170]"'}


In [37]:
  
if response.status_code == 200:
    
    # Assuming the server response is already parsed into a JSON-like dict format
    cleanedjson = parse_coordinates(server_response["body"])
    print("Cleaned JSON:",cleanedjson)
    
    if(cleanedjson == {"Image": []}):
        print("No se encontraron botellas en la imagen")
    else:
        print("Botellas encontradas en la imagen:", len(cleanedjson["Image"]))
        result_image_path = draw_lines_on_image_base64(encoded_image, cleanedjson)
        
    print(f"Result image saved at {result_image_path}")
else:
    print("Error en la solicitud:", response.status_code, response.text)
    
print("Fin de la ejecución")

Cleaned JSON: {'Image': [{'bottle': 1, 'coordinates': {'upper': [101, 90], 'lower': [140, 170]}}, {'bottle': 2, 'coordinates': {'upper': [141, 90], 'lower': [180, 170]}}, {'bottle': 3, 'coordinates': {'upper': [181, 90], 'lower': [220, 170]}}, {'bottle': 4, 'coordinates': {'upper': [221, 90], 'lower': [260, 170]}}, {'bottle': 5, 'coordinates': {'upper': [261, 90], 'lower': [300, 170]}}, {'bottle': 6, 'coordinates': {'upper': [301, 90], 'lower': [340, 170]}}, {'bottle': 7, 'coordinates': {'upper': [341, 90], 'lower': [380, 170]}}, {'bottle': 8, 'coordinates': {'upper': [381, 90], 'lower': [420, 170]}}]}
Botellas encontradas en la imagen: 8
Result image saved at result_image.png
Fin de la ejecución
